# 何謂 System-Versioned Temporal Table

# 針對現有的資料表，加上 Temporal Table 的功能

### 建立不含 Temporal Table 的資料表， 同時新增 2 筆資料。

In [ ]:
USE TemporalSample
GO

In [ ]:
DROP TABLE IF EXISTS [dbo].[Department]
GO

In [ ]:
CREATE TABLE [dbo].[Department]
( [ID]     INT           NOT NULL,
  [Name]   NVARCHAR(20)  NOT NULL CONSTRAINT PK_Department PRIMARY KEY CLUSTERED ( [ID] )
)
GO

In [ ]:
INSERT INTO [dbo].[Department]
VALUES   (1, 'MIS'),
         (2, 'HR')
GO

In [ ]:
SELECT *
FROM [dbo].[Department];
GO

### 為資料表 Department 加上 Temporal Table 的功能。

In [ ]:
ALTER TABLE [dbo].[Department] 
ADD
  ValidFrom DATETIME2 (2) GENERATED ALWAYS AS ROW START HIDDEN  
        CONSTRAINT DF_ValidFrom DEFAULT DATEADD(SECOND, -1, SYSUTCDATETIME()),
  ValidTo DATETIME2 (2)  GENERATED ALWAYS AS ROW END HIDDEN   
        CONSTRAINT DF_ValidTo DEFAULT '9999.12.31 23:59:59.99',
  PERIOD FOR SYSTEM_TIME (ValidFrom, ValidTo)
GO

In [ ]:
ALTER TABLE [dbo].[Department]   
    SET (SYSTEM_VERSIONING = ON (HISTORY_TABLE = dbo.DepartmentHistory))
GO

In [ ]:
SELECT *
FROM [dbo].[Department]
GO

In [ ]:
SELECT ID, Name, ValidFrom, ValidTo
FROM [dbo].[Department]
GO

# 建立新的資料表，同時加上 Temporal Table 的功能

In [ ]:
USE TemporalSample
GO

---- Code Block 1 ---- (BEGIN)

IF OBJECT_ID('dbo.Employee') IS NOT NULL
   ALTER TABLE [dbo].[Employee] SET ( SYSTEM_VERSIONING = OFF) 
GO

DROP TABLE IF EXISTS [dbo].[Employee]
GO

DROP TABLE IF EXISTS [dbo].[EmployeeHistory]
GO

---- Code Block 1 ---- (END)
 

CREATE TABLE Employee 
(  
   [ID]         INT NOT NULL  PRIMARY KEY CLUSTERED,
   [Name]       NVARCHAR(50)  NOT NULL,
   [Position]   NVARCHAR(50)  NOT NULL,
   [DeptID]     INT           NOT NULL,
   [Address]    NVARCHAR(100) NOT NULL,
   [AnnualSalary] DECIMAL(10,2) NOT NULL,
   [ValidFrom]  DATETIME2 (2) GENERATED ALWAYS AS ROW START,
   [ValidTo]    DATETIME2 (2) GENERATED ALWAYS AS ROW END,
   PERIOD FOR SYSTEM_TIME (ValidFrom, ValidTo)
)  
WITH (SYSTEM_VERSIONING = ON (HISTORY_TABLE = dbo.EmployeeHistory))
GO

In [ ]:
IF OBJECT_ID('dbo.Employee') IS NOT NULL
   ALTER TABLE [dbo].[Employee] SET ( SYSTEM_VERSIONING = OFF)
GO

In [ ]:
ALTER TABLE [dbo].[Employee]  
    SET (SYSTEM_VERSIONING = ON (HISTORY_TABLE = dbo.EmployeeHistory))
GO

# 新增 2 筆資料

In [ ]:
USE [TemporalSample]
GO

In [ ]:
INSERT INTO [dbo].[Employee]
   ([ID], [Name], [Position], [DeptID], [Address], [AnnualSalary] )
VALUES
   (1, 'jasper', 'engineer', 1, 'taipei', 1500),
   (2, 'joseph', 'manager', 1, 'taipei',  3000)
GO

In [ ]:
SELECT *
FROM [dbo].[Employee]
GO

SELECT *
FROM [dbo].[EmployeeHistory]
GO

# 修改 1 筆資料

In [ ]:
USE TemporalSample
GO

UPDATE A
SET A.[AnnualSalary] = 4000
FROM [dbo].[Employee] A
WHERE A.[ID] = 2
GO

# 刪除 1 筆資料

In [ ]:
USE TemporalSample
GO

DELETE A
FROM [dbo].[Employee] A
WHERE A.[ID] = 2
GO

# 查 Temporal Table 的一些語法

In [ ]:
USE [TemporalSample]
GO

In [ ]:
SELECT [ID],
       [Name],
       [Position],
       [DeptID],
       [Address],
       [AnnualSalary],
       [ValidFrom],
       [ValidTo],
  FROM [dbo].[Employee]

FOR SYSTEM_TIME ALL ORDER BY [ValidFrom]
GO

In [ ]:
SELECT [ID],
       [Name],
       [Position],
       [DeptID],
       [Address],
       [AnnualSalary],
       [ValidFrom],
       [ValidTo],
  FROM [dbo].[Employee]

FOR SYSTEM_TIME BETWEEN '2017-12-07 T06:35:00' AND '2017-12-08 T00:00:00'
GO

In [ ]:
SELECT [ID],
       [Name],
       [Position],
       [DeptID],
       [Address],
       [AnnualSalary],
       [ValidFrom],
       [ValidTo],
  FROM [dbo].[Employee]

FOR SYSTEM_TIME FROM '2017-12-07 T06:35:00' TO '2017-12-08 T00:00:00'
GO

In [ ]:
SELECT [ID],
       [Name],
       [Position],
       [DeptID],
       [Address],
       [AnnualSalary],
       [ValidFrom],
       [ValidTo],
  FROM [dbo].[Employee]

FOR SYSTEM_TIME CONTAINED IN ('2017-12-07 T06:35:00','2017-12-08 T00:00:00')
GO

In [ ]:
SELECT [ID],
       [Name],
       [Position],
       [DeptID],
       [Address],
       [AnnualSalary],
       [ValidFrom],
       [ValidTo],
  FROM [dbo].[Employee]

FOR SYSTEM_TIME CONTAINED IN ('2017-12-07 T00:00:00','2017-12-08 T00:00:00')
GO

In [ ]:
SELECT [ID],
       [Name],
       [Position],
       [DeptID],
       [Address],
       [AnnualSalary],
       [ValidFrom],
       [ValidTo],
  FROM [dbo].[Employee]

FOR SYSTEM_TIME AS OF '2017-12-07 06:33:00.00'
GO

### 範例01-修復資料列層級資料損毀

參考文章: https://docs.microsoft.com/zh-tw/sql/relational-databases/tables/temporal-table-usage-scenarios?view=sql-server-ver16

In [ ]:
CREATE TABLE Employee
(
  [EmployeeID] int NOT NULL PRIMARY KEY CLUSTERED,
  [Name] nvarchar(100) NOT NULL,
  [Position] varchar(100) NOT NULL,
  [Department] varchar(100) NOT NULL,
  [Address] nvarchar(1024) NOT NULL,
  [AnnualSalary] decimal(10,2) NOT NULL,
  [ValidFrom] datetime2(2) GENERATED ALWAYS AS ROW START,
  [ValidTo] datetime2(2) GENERATED ALWAYS AS ROW END,
  PERIOD FOR SYSTEM_TIME (ValidFrom, ValidTo)
 )
 WITH (SYSTEM_VERSIONING = ON (HISTORY_TABLE = dbo.EmployeeHistory))

In [ ]:
DROP PROCEDURE IF EXISTS RepairEmployeeRecord
GO

CREATE PROCEDURE RepairEmployeeRecord
    @EmployeeID INT,
    @versionNumber INT = 1
AS

WITH History
AS
(
        /* Order historical rows by their age in DESC order*/
        SELECT ROW_NUMBER () OVER (PARTITION BY EmployeeID ORDER BY [ValidTo] DESC) AS RN, *
        FROM Employee FOR SYSTEM_TIME ALL WHERE YEAR(ValidTo) < 9999 AND Employee.EmployeeID = @EmployeeID
)

/*Update current row by using N-th row version from history (default is 1 - i.e. last version)*/
UPDATE Employee
    SET [Position] = H.[Position], [Department] = H.Department, [Address] = H.[Address], AnnualSalary = H.AnnualSalary
    FROM Employee E JOIN History H ON E.EmployeeID = H.EmployeeID AND RN = @versionNumber
    WHERE E.EmployeeID = @EmployeeID

In [ ]:
EXEC RepairEmployeeRecord @EmployeeID = 2, @versionNumber = 1